In [30]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset, DatasetDict
from collections import Counter
from wordcloud import WordCloud
from unidecode import unidecode

In [2]:
data = load_dataset("qanastek/MASSIVE", trust_remote_code=True)
print(data)

DatasetDict({
    train: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'tokens', 'ner_tags', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 587214
    })
    validation: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'tokens', 'ner_tags', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 103683
    })
    test: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'tokens', 'ner_tags', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 151674
    })
})


In [28]:
des_col = ['locale', 'partition', 'utt', 'tokens']

data1 = DatasetDict({
    'train': data['train'].select_columns(des_col),
    'validation': data['validation'].select_columns(des_col),
    'test': data['test'].select_columns(des_col)
})

print(data1)

DatasetDict({
    train: Dataset({
        features: ['locale', 'partition', 'utt', 'tokens'],
        num_rows: 587214
    })
    validation: Dataset({
        features: ['locale', 'partition', 'utt', 'tokens'],
        num_rows: 103683
    })
    test: Dataset({
        features: ['locale', 'partition', 'utt', 'tokens'],
        num_rows: 151674
    })
})


In [ ]:
train_data = data1['train']
val_data = data1['validation']
test_data = data1['test']

In [32]:
locales = ['af-ZA', 'da-DK', 'de-DE', 'en-US', 'es-ES', 'fr-FR', 'fi-FI', 'hu-HU', 'is-IS', 
           'it-IT', 'jv-ID', 'lv-LV', 'ms-MY', 'nb-NO', 'nl-NL', 'pl-PL', 'pt-PT', 
           'ro-RO', 'ru-RU', 'sl-SL', 'sv-SE', 'sq-AL', 'sw-KE', 'tl-PH', 'tr-TR', 
           'vi-VN', 'cy-GB']

# Directory to store output files
output_dir = "utts_by_locale"
os.makedirs(output_dir, exist_ok=True)

def extr_utt(dataset, locales, output_dir):
    for locale in locales:
        locale_data = dataset.filter(lambda example: example['locale'] == locale)
        
        file_path = os.path.join(output_dir, f"{locale}.txt")
        with open(file_path, 'w', encoding='utf-8') as file:
            for utt in locale_data['utt']:
                file.write(utt + "\n")                            # 1 utterance per line

        print(f"Saved {locale} utterances to {file_path}")

In [33]:
extr_utt(data1['train'], locales, output_dir)

Filter: 100%|██████████| 587214/587214 [00:12<00:00, 48297.02 examples/s]


Saved af-ZA utterances to utts_by_locale\af-ZA.txt


Filter: 100%|██████████| 587214/587214 [00:15<00:00, 37591.11 examples/s]


Saved da-DK utterances to utts_by_locale\da-DK.txt


Filter: 100%|██████████| 587214/587214 [00:16<00:00, 35147.14 examples/s]


...

Saved tr-TR utterances to utts_by_locale\tr-TR.txt


Filter: 100%|██████████| 587214/587214 [00:18<00:00, 32533.02 examples/s]


Saved vi-VN utterances to utts_by_locale\vi-VN.txt


Filter: 100%|██████████| 587214/587214 [00:17<00:00, 32844.89 examples/s]


Saved cy-GB utterances to utts_by_locale\cy-GB.txt
